### 

### Reading data into spark data frame

In [1]:
path = "abfss://fscustransdevnorthcent@stcustransdevnorthcent.dfs.core.windows.net/silver/upcsilver_raw/"
sdfUPC = spark.read.parquet(path)

### Creating a temp view based on the sdfUPC data frame

In [2]:
sdfUPC.createOrReplaceTempView("vwUPC_raw")

### 

### Creating a temporary view based on vwUPC_raw with modifications
Added status codes and returned only distinct records in the data set

In [3]:
%%sql
CREATE OR REPLACE TEMP VIEW vwUPC
AS

WITH cteUPC AS (
    SELECT 
         u.PolicyNumber
        ,u.Insured
        ,u.Description
        ,u.DateSent
        ,CASE
            WHEN description = 'Dec Page' THEN 'NEW'
            WHEN description = 'Amended Dec Page' THEN 'AMENDED'
            WHEN description = 'Renewal Offer' THEN 'OFFER'
            WHEN description = 'Notice of Lapsed Policy' THEN 'LAPSED'
            WHEN description = 'Notice of Reinstatement' THEN 'REINSTATED'
            WHEN description = 'Endorsement Form' THEN 'UPDATE'
            WHEN description = 'Cancellation Letter' THEN 'CANCELLED'
            WHEN description = 'Balance Due' THEN 'BALANCE DUE'
            WHEN description = 'Underwriting Information Request Form' THEN 'PENDING'
            WHEN description = 'Underwriting Standards Form' THEN 'AT RISK'
            WHEN description = 'AOR(agent of record) change letter' THEN 'NON-RENEWAL-AOR'
            WHEN description = 'Binder Cancellation' THEN 'AT RISK'
        END AS Status
    FROM vwUPC_raw AS u
)

SELECT DISTINCT u.* from cteUPC AS u

In [4]:
%%sql
USE silver

In [5]:
%%sql 
INSERT INTO combine_data(agency_number, insured, company, date, policy_number, status)
SELECT 
'21266' AS agency_number
,insured
,'UPC' AS company
,date_format(to_date(datesent, 'MM/dd/yyyy'),'yyyyMMdd') as date
,PolicyNumber AS policy_number
,status
FROM vwUPC
